In [ ]:
import pandas as pd

# download from https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset/data

movies = pd.read_csv('dataset/movies_metadata.csv', low_memory=False)
movies.shape

In [ ]:
movies.columns

In [ ]:
from math import isnan
from pprint import pprint

trimmed_movies = movies[["title", "overview", "release_date", "genres"]]
trimmed_movies.head(4)

unclean_movie_dict = trimmed_movies.to_dict('records')
print('{} movies'.format(len(unclean_movie_dict)))

movies_dict = []

for movie in unclean_movie_dict:
    if movie["overview"] == movie["overview"] and movie["release_date"] == movie["release_date"] and movie["genres"] == movie["genres"]:
        movies_dict.append(movie)

print('{} movies'.format(len(movies_dict)))

In [ ]:
from pymilvus import *

milvus_uri = "http://192.168.1.106:19530"

connections.connect(uri=milvus_uri)

print("Connected!")

In [ ]:
from Milvus import FieldSchema, DataType, CollectionSchema, utility, Collection

COLLECTION_NAME = 'film_vectors'
PARTITION_NAME = 'movie'

id = FieldSchema(name='title', dtype=DataType.VARCHAR, max_length=500, is_primary=True)
field = FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, dim=384)


schema = CollectionSchema(field=[id, field], description="movie recommender: film vectors", enable_dynamic_field=True)

if utility.has_collection(COLLECTION_NAME):
    collection = Collection(COLLECTION_NAME)
    collection.drop()

collection = Collection(COLLECTION_NAME, schema)
print("Collection created.")

index_params = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist":128}
}

collection.create_index(field_name="embedding", index_params=index_params)
collection.load()

print("Collection indexed!")

In [1]:
from sentence_transformers import SentenceTransformer
import ast

print('Done')
